In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Weather Loader").getOrCreate()
print(spark)

In [ ]:
historic_df = spark.read.csv("/content/drive/MyDrive/air_quality_final_20250929_105402.csv", header=True, inferSchema=True)

# Filter out rows with year less than 2000 and missing aqi values
historic_df = historic_df.filter((historic_df.year >= 2000) & (historic_df.aqi.isNotNull()))

# Drop the specified columns
historic_df = historic_df.drop("_c21", "geocoding_source")

historic_df.show()

+--------------+----------------+----------------------+------------------------------+--------------------+----+----+---------+------+----+-----+---+-------------------+----+------+------+---+------------+----------+----------+
|_sampling_date|           state|city_town_village_area|location_of_monitoring_station|    type_of_location| so2| no2|rspm_pm10|pm_2_5|year|month|day|               time|hour|minute|second|aqi|aqi_category|  latitude| longitude|
+--------------+----------------+----------------------+------------------------------+--------------------+----+----+---------+------+----+-----+---+-------------------+----+------+------+---+------------+----------+----------+
|7/1/2014 12:48|       Meghalaya|                 Dawki|          Terrace building,...|Residential, Rura...| 2.0|11.0|     52.0|  NULL|2014|    7|  1|2025-10-03 12:48:04|  12|    48|     4| 52|Satisfactory|25.1856343|92.0215717|
|1/1/2014 12:25|             Goa|                Panaji|          Infront of Old GS.

In [ ]:
from pyspark.sql.functions import to_timestamp

historic_df = historic_df.withColumn("datetime", to_timestamp(historic_df["_sampling_date"], "M/d/yyyy H:m"))

historic_df.show()

+--------------+----------------+----------------------+------------------------------+--------------------+----+----+---------+------+----+-----+---+-------------------+----+------+------+---+------------+----------+----------+-------------------+
|_sampling_date|           state|city_town_village_area|location_of_monitoring_station|    type_of_location| so2| no2|rspm_pm10|pm_2_5|year|month|day|               time|hour|minute|second|aqi|aqi_category|  latitude| longitude|           datetime|
+--------------+----------------+----------------------+------------------------------+--------------------+----+----+---------+------+----+-----+---+-------------------+----+------+------+---+------------+----------+----------+-------------------+
|7/1/2014 12:48|       Meghalaya|                 Dawki|          Terrace building,...|Residential, Rura...| 2.0|11.0|     52.0|  NULL|2014|    7|  1|2025-10-03 12:48:04|  12|    48|     4| 52|Satisfactory|25.1856343|92.0215717|2014-07-01 12:48:00|
|1/1

In [ ]:
historic_df = historic_df.drop("_sampling_date","second","time")

In [ ]:
# Get unique location-year pairs using Spark DataFrame operations
unique_loc_year_spark = historic_df.select('latitude', 'longitude', 'year').distinct()

# Show the count of unique location-year pairs
print("Total unique (lat,lon,year) pairs:", unique_loc_year_spark.count())

# Display the first few rows of the unique location-year pairs
unique_loc_year_spark.show(n=5)

Total unique (lat,lon,year) pairs: 2495
+----------+----------+----+
|  latitude| longitude|year|
+----------+----------+----+
| 17.360589|78.4740613|2014|
|23.3701968|85.3249027|2004|
|25.6618755|94.1019156|2011|
|26.9153191|75.8169299|2012|
|23.1883478|79.9092722|2005|
+----------+----------+----+
only showing top 5 rows



In [ ]:
import os
import pandas as pd
import requests
import time
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, TimestampType

# Initialize Spark session
spark = SparkSession.builder.appName("WeatherFetcher").getOrCreate()

# Schema for final Spark DataFrame
weather_schema = StructType([
    StructField("datetime", TimestampType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("wind", DoubleType(), True),
    StructField("humidity", DoubleType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True)
])

# ----------------------------
# Function to fetch weather
# ----------------------------
def fetch_weather(lat, lon, year, max_retries=5):
    lat = float(lat)
    lon = float(lon)
    year = int(year)

    start = f"{year}-01-01"
    end   = f"{year}-12-31"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start,
        "end_date": end,
        "hourly": ["temperature_2m", "windspeed_10m", "relative_humidity_2m"],
        "timezone": "Asia/Kolkata"
    }

    retries = 0
    while retries < max_retries:
        try:
            r = requests.get("https://archive-api.open-meteo.com/v1/archive", params=params, timeout=60)
            r.raise_for_status()
            data = r.json()

            if "hourly" not in data or not data["hourly"]["time"]:
                return None

            pdf = pd.DataFrame({
                "datetime": pd.to_datetime(data["hourly"]["time"]),
                "temperature": data["hourly"]["temperature_2m"],
                "wind": data["hourly"]["windspeed_10m"],
                "humidity": data["hourly"]["relative_humidity_2m"],
                "lat": lat,
                "lon": lon
            })
            return pdf

        except (requests.exceptions.RequestException, json.JSONDecodeError):
            retries += 1
            time.sleep(2 ** retries)  # exponential backoff
        except Exception as e:
            print(f"Unexpected error for {lat},{lon},{year}: {e}")
            return None
    return None

# ----------------------------
# MAIN PIPELINE
# ----------------------------
# Get your unique (lat, lon, year) pairs from Spark
pairs = unique_loc_year_spark.collect()
print(f"Total tasks: {len(pairs)}")

results = []

# Run parallel fetching
with ThreadPoolExecutor(max_workers=20) as executor:  # adjust workers depending on CPU/internet
    futures = {
        executor.submit(fetch_weather, row.latitude, row.longitude, row.year): row
        for row in pairs
    }

    for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching weather"):
        pdf = future.result()
        if pdf is not None:
            results.append(pdf)

# ----------------------------
# Convert results to Spark DataFrame
# ----------------------------
if results:
    all_weather_pd = pd.concat(results, ignore_index=True)
    weather_df = spark.createDataFrame(all_weather_pd, schema=weather_schema)

    # Save fewer parquet files
    weather_df.coalesce(10).write.mode("overwrite").parquet(
        "C:/Users/SNEHIL/Downloads/Air_quality_Monitoring/final_weather_data_all"
    )
    print("✅ Weather data saved successfully!")
else:
    print("⚠️ No data fetched.")


In [ ]:
weather_df = spark.read.parquet('/content/drive/MyDrive/all_weather_pd_compatible_final.parquet')

In [ ]:
weather_df.show()

+-------------------+-----------+----+--------+---------+----------+
|           datetime|temperature|wind|humidity|      lat|       lon|
+-------------------+-----------+----+--------+---------+----------+
|2013-01-01 00:00:00|       25.2| 7.1|      89|9.5006651|76.4124143|
|2013-01-01 01:00:00|       25.1| 6.0|      89|9.5006651|76.4124143|
|2013-01-01 02:00:00|       24.7| 4.7|      91|9.5006651|76.4124143|
|2013-01-01 03:00:00|       24.3| 3.5|      93|9.5006651|76.4124143|
|2013-01-01 04:00:00|       24.1| 4.6|      94|9.5006651|76.4124143|
|2013-01-01 05:00:00|       23.9| 4.1|      94|9.5006651|76.4124143|
|2013-01-01 06:00:00|       23.6| 3.5|      96|9.5006651|76.4124143|
|2013-01-01 07:00:00|       24.2| 4.7|      95|9.5006651|76.4124143|
|2013-01-01 08:00:00|       26.1| 5.4|      85|9.5006651|76.4124143|
|2013-01-01 09:00:00|       27.2| 4.5|      79|9.5006651|76.4124143|
|2013-01-01 10:00:00|       28.4| 2.6|      73|9.5006651|76.4124143|
|2013-01-01 11:00:00|       29.4| 

In [ ]:
from pyspark.sql.functions import date_trunc

# Truncate the datetime column in historic_df to the hour
historic_df_hourly = historic_df.withColumn("datetime_hourly", date_trunc("hour", historic_df["datetime"]))

# Select only the desired columns from weather_df
weather_df_selected = weather_df.select("datetime", "temperature", "wind", "humidity", "lat", "lon")

# Join the two dataframes on latitude, longitude, and the hour part of the datetime
joined_df = historic_df_hourly.join(
    weather_df_selected,
    (historic_df_hourly.latitude == weather_df_selected.lat) &
    (historic_df_hourly.longitude == weather_df_selected.lon) &
    (historic_df_hourly.datetime_hourly == weather_df_selected.datetime),
    "inner"
)

# Drop the redundant latitude, longitude and datetime_hourly columns from historic_df_hourly
joined_df = joined_df.drop(historic_df_hourly.latitude).drop(historic_df_hourly.longitude).drop("datetime_hourly")


joined_df.show()

+------+----------------------+------------------------------+--------------------+---+----+---------+------+----+-----+---+----+------+---+------------+-------------------+-------------------+-----------+----+--------+---------+----------+
| state|city_town_village_area|location_of_monitoring_station|    type_of_location|so2| no2|rspm_pm10|pm_2_5|year|month|day|hour|minute|aqi|aqi_category|           datetime|           datetime|temperature|wind|humidity|      lat|       lon|
+------+----------------------+------------------------------+--------------------+---+----+---------+------+----+-----+---+----+------+---+------------+-------------------+-------------------+-----------+----+--------+---------+----------+
|Kerala|            Trivandrum|                  PRS Hospital|Residential and o...|7.1|23.2|     63.0|  NULL|2005|    6| 21|   4|    45| 63|Satisfactory|2005-06-21 04:45:00|2005-06-21 04:00:00|       23.8|12.0|      95|8.4811163|76.9592286|
|Kerala|            Trivandrum|     

In [ ]:
from pyspark.sql.functions import col

missing_temperature_count = joined_df1.filter(col("temperature").isNull()).count()

print(f"Number of times temperature is missing: {missing_temperature_count}")

Number of times temperature is missing: 0


In [ ]:
from pyspark.sql.functions import col

# Filter for rows where temperature is missing
missing_temperature_df = joined_df.filter(col("temperature").isNull())

# Select unique pairs of latitude, longitude, and year
unique_missing_temp_locations = missing_temperature_df.select('latitude', 'longitude', 'year').distinct()

# Show the unique pairs
unique_missing_temp_locations.show()

+--------+---------+----+
|latitude|longitude|year|
+--------+---------+----+
+--------+---------+----+



In [ ]:
# Drop the redundant datetime column from weather_df_selected
joined_df1 = joined_df.drop(weather_df_selected.datetime)

joined_df.printSchema()

root
 |-- state: string (nullable = true)
 |-- city_town_village_area: string (nullable = true)
 |-- location_of_monitoring_station: string (nullable = true)
 |-- type_of_location: string (nullable = true)
 |-- so2: double (nullable = true)
 |-- no2: double (nullable = true)
 |-- rspm_pm10: double (nullable = true)
 |-- pm_2_5: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- aqi: integer (nullable = true)
 |-- aqi_category: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- datetime: timestamp_ntz (nullable = true)
 |-- temperature: double (nullable = true)
 |-- wind: double (nullable = true)
 |-- humidity: long (nullable = true)



In [ ]:
joined_df1.show(n=5)

+------+----------------------+------------------------------+--------------------+---+----+---------+------+----+-----+---+----+------+---+------------+-------------------+-----------+----+--------+
| state|city_town_village_area|location_of_monitoring_station|    type_of_location|so2| no2|rspm_pm10|pm_2_5|year|month|day|hour|minute|aqi|aqi_category|           datetime|temperature|wind|humidity|
+------+----------------------+------------------------------+--------------------+---+----+---------+------+----+-----+---+----+------+---+------------+-------------------+-----------+----+--------+
|Kerala|            Trivandrum|                  PRS Hospital|Residential and o...|7.1|23.2|     63.0|  NULL|2005|    6| 21|   4|    45| 63|Satisfactory|2005-06-21 04:45:00|       23.8|12.0|      95|
|Kerala|            Trivandrum|                  PRS Hospital|Residential and o...|6.4|25.0|     65.0|  NULL|2005|    6| 24|   4|    45| 65|Satisfactory|2005-06-24 04:45:00|       24.0|15.3|      95|


In [ ]:
# Drop the redundant latitude, longitude and datetime_hourly columns from historic_df_hourly
joined_df = joined_df.drop( "lat", "lon")
joined_df.show()

+------+----------------------+------------------------------+--------------------+---+----+---------+------+----+-----+---+----+------+---+------------+-------------------+-------------------+-----------+----+--------+
| state|city_town_village_area|location_of_monitoring_station|    type_of_location|so2| no2|rspm_pm10|pm_2_5|year|month|day|hour|minute|aqi|aqi_category|           datetime|           datetime|temperature|wind|humidity|
+------+----------------------+------------------------------+--------------------+---+----+---------+------+----+-----+---+----+------+---+------------+-------------------+-------------------+-----------+----+--------+
|Kerala|            Trivandrum|                  PRS Hospital|Residential and o...|7.1|23.2|     63.0|  NULL|2005|    6| 21|   4|    45| 63|Satisfactory|2005-06-21 04:45:00|2005-06-21 04:00:00|       23.8|12.0|      95|
|Kerala|            Trivandrum|                  PRS Hospital|Residential and o...|6.4|25.0|     65.0|  NULL|2005|    6|

In [ ]:
joined_df1.write.parquet("/content/drive/MyDrive/Air Quality Monitoring Data/ processed / govt_data.parquet")

In [ ]:
df1 = spark.read.parquet("/content/drive/MyDrive/Air Quality Monitoring Data/ processed /air_quality_with_weather1.parquet")

In [ ]:
df1.printSchema()

root
 |-- location_id: long (nullable = true)
 |-- sensors_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- datetime: timestamp_ntz (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- value_co: double (nullable = true)
 |-- value_no2: double (nullable = true)
 |-- value_o3: double (nullable = true)
 |-- value_pm10: double (nullable = true)
 |-- value_pm25: double (nullable = true)
 |-- value_so2: double (nullable = true)
 |-- datetime_hour: timestamp_ntz (nullable = true)
 |-- lat_round: double (nullable = true)
 |-- lon_round: double (nullable = true)
 |-- temperature: double (nullable = true)
 |-- wind: double (nullable = true)
 |-- humidity: long (nullable = true)
 |-- units: string (nullable = true)
 |-- AQI: double (nullable = true)

